In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import pandas
import csv
from PIL import Image
import sys
import random as rnd
import copy

%run ./img_util.py

In [3]:
emotion_table = {'neutral'  : 0, 
                 'happiness': 1, 
                 'surprise' : 2, 
                 'sadness'  : 3, 
                 'anger'    : 4, 
                 'disgust'  : 5, 
                 'fear'     : 6, 
                 'contempt' : 7}
base_path = os.path.join(".", "fer2013")

In [4]:
train_folders = os.path.join(base_path,'FER2013Train')
valid_folders = os.path.join(base_path,'FER2013Valid') 
test_folders  = os.path.join(base_path,'FER2013Test')

In [5]:
def process_data(emotion_raw, mode="mayority"):
    '''
    Based on https://arxiv.org/abs/1608.01041, we process the data differently depend on the training mode:
    Majority: return the emotion that has the majority vote, or unknown if the count is too little.
    Crossentropty: convert the count into probability distribution.abs
    '''        
    size = len(emotion_raw)
    emotion_unknown     = [0.0] * size
    emotion_unknown[-2] = 1.0

    # remove emotions with a single vote (outlier removal) 
    for i in range(size):
        if emotion_raw[i] < 1.0 + sys.float_info.epsilon:
            emotion_raw[i] = 0.0

    sum_list = sum(emotion_raw)
    emotion = [0.0] * size 

    if mode == 'majority': 
        # find the peak value of the emo_raw list 
        maxval = max(emotion_raw) 
        if maxval > 0.5*sum_list: 
            emotion[np.argmax(emotion_raw)] = maxval 
        else: 
            emotion = emotion_unknown   # force setting as unknown 
    elif (mode == 'crossentropy'):
        sum_part = 0
        count = 0
        valid_emotion = True
        while sum_part < 0.75*sum_list and count < 3 and valid_emotion:
            maxval = max(emotion_raw) 
            for i in range(size): 
                if emotion_raw[i] == maxval: 
                    emotion[i] = maxval
                    emotion_raw[i] = 0
                    sum_part += emotion[i]
                    count += 1
                    if i >= 8:  # unknown or non-face share same number of max votes 
                        valid_emotion = False
                        if sum(emotion) > maxval:   # there have been other emotions ahead of unknown or non-face
                            emotion[i] = 0
                            count -= 1
                        break
        if sum(emotion) <= 0.5*sum_list or count > 3: # less than 50% of the votes are integrated, or there are too many emotions, we'd better discard this example
            emotion = emotion_unknown   # force setting as unknown 
            
    return [float(i)/sum(emotion) for i in emotion]

In [6]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [7]:
def load_data(folder, mode):
    names = []
    data = []
    labels = []
    per_emotion_count = np.zeros(len(emotion_table), dtype=np.int)
    in_label_path = os.path.join(folder, "label.csv")
    with open(in_label_path) as csvfile: 
        emotion_label = csv.reader(csvfile) 
        for row in emotion_label:
            # load the image
            image_path = os.path.join(folder, row[0])
            img_data = Image.open(image_path)
            rgb_im = img_data.convert("RGB")
            
            rotated_image = rgb_im.transpose(Image.FLIP_LEFT_RIGHT)
            
            rgb_im.load()
            rotated_image.load()
                        
            emotion_raw = list(map(float, row[2:len(row)]))
            emotion = process_data(emotion_raw, mode)
            
            idx = np.argmax(emotion)
            if idx < len(emotion_table): # not unknown or non-face 
                emotion = emotion[:-2]
                emotion = [float(i)/sum(emotion) for i in emotion]
                names.append(image_path)
                names.append("rotated_"+image_path)
                data.append(np.array(rotated_image).tolist())
                data.append(np.array(rgb_im).tolist())
                labels.append(emotion)
                labels.append(emotion)
                per_emotion_count[idx] += 1
    return names, data, labels

In [8]:
_, x_train, y_train = load_data(train_folders, "crossentropy") #Train Data

In [9]:
_, x_valid, y_valid = load_data(valid_folders, "majority") #Valid Data

In [10]:
_, x_test, y_test = load_data(test_folders, "majority") #Test Data

In [16]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow(np.array(x_train),
                                 y=y_train,
                                 batch_size=32,
                                 seed=256)

valid_set = train_datagen.flow(np.array(x_valid),
                                 y=y_valid,
                                 batch_size=32,
                                 seed=256)

In [11]:
from keras.applications.vgg16 import VGG16
from keras.optimizers import Adamax

In [12]:
vgg16 = VGG16(include_top=False, input_shape=(48, 48, 3), weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
model = Sequential()

for layer in vgg16.layers:
    model.add(layer)

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128,activation='relu'))
model.add(Dense(len(emotion_table), activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
adamax = Adamax()

model.compile(loss='categorical_crossentropy',
              optimizer=adamax, metrics=['accuracy'])

In [15]:
#_, x_train, y_train = np.transpose(np.array(train_data))
#_, x_valid, y_valid = np.transpose(np.array(valid_data))
#_, x_test, y_test = np.transpose(np.array(test_data))

In [ ]:
model.fit_generator(training_set,
                         steps_per_epoch = len(x_train)/32,
                         epochs = 25,
                         validation_data = valid_set,
                         validation_steps = len(x_valid)/32)

Instructions for updating:
Use tf.cast instead.
Epoch 1/25
 218/1746 [==>...........................] - ETA: 1:37:25 - loss: 11.6462 - acc: 0.2787

In [ ]:
classifier.save('trial2.h5')

In [ ]:
f= open("trial2.json","w+")
f.write(str(classifier.history.history))
f.close()